In [5]:
# 필요한 라이브러리 불러오기

## 데이터 분석 관련
import pandas as pd
from pandas import Series, DataFrame
import numpy as np

## 데이터 시각화 관련
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid') #matplotlib의 스타일에 관련 스타일
## 그래프 출력에 필요한 IPython 명령어
%matplotlib inline

## Scikit-Learn 의 다양한 머신러닝 모듈을 불러온다.
## 분류 알고리즘 중에서 선형회귀, 서포트벡터머신, 랜덤포레스트, K-최근점이웃 알고리즘을 사용해본다.
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [89]:
# 데이터 가져오기
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# 데이터 미리보기
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [90]:
# 데이터 살펴보기
train_df.info()
print('-'*20)
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 66.2+ KB
--------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    

### pandas의 info() 메서드
DataFrame을 간결하게 요약하여 출력하며, 인덱스의 타입, 열, null이 아닌 값, 메모리 사용량을 출력한다.

## 위 결과에서 알 수 있는 사항
* train 데이터프레임에서는 Age와 Cabin, Embarked가 비어있는 값이 있고, test 데이터프레임에서는 Age, Fare, Cabin에 비어있는 값이 있다는 것을 알 수 있다.
* train 데이터프레임의 특성은 12개, test 데이터프레임의 특성은 11개 인 것을 알 수 있고 test 데이터프레임에는 Survived 특성이 없다.

## 처리해주어야할 사항
* 데이터의 빈 부분을 어떻게 처리할 것인가?
* 데이터를 float64 형태로 변환할 수 있는지?(모델에 들어가기 위해서는 float64 형태이어야 한다.)

In [91]:
# train_df = train_df.drop(['PassengerId', 'Name', 'Ticket'], axis = 1)
# test_df = test_df.drop(['Name', 'Ticket'], axis = 1)
train_df = train_df.drop(['PassengerId', 'Name', 'Ticket'], axis=1)
test_df = test_df.drop(['Name','Ticket'], axis=1)

### pandas의 drop() 메서드
- 열이나 행의 라벨 값으로 값을 지우는 메서드이다.
- axis = 0 이면 인덱스(행)을 지우고 axis = 1 이면 컬럼(열)을 지운다.
- 지운 dataframe 을 반환

## test_df에서는 PassengerId를 지우지 않는 이유
Evaluation 과정에서 결과를 csv 파일로 제출해야하는데 이때 반드시 PassengerId와 Survived 열이 있어야 한다.

# 1. Pclass 데이터 처리하기
- 1등석, 2등석, 3등석인지 정보로 처음에 데이터가 비어있지 않았다는 것을 알 수 있었다.
- int 자료형이다.

In [92]:
train_df['Pclass'].value_counts()

3    491
1    216
2    184
Name: Pclass, dtype: int64

### pandas의 value_counts(subset = None, normalize = False, sort = True, ascending = False) 메서드
- 데이터프레임에서 행을 분류별로 세서 Series로 반환한다.
- normalize = True로 하면 비율로 출력할 수 있다.

## Pclass
1, 2, 3이 정수라서 실수로만 바꾸면 되는 것이 아니다. 이는 단순히 숫자로 분류된 것일 뿐 연속적인 정보가 아니고 각 차이가 균등하지 않다. 그렇기 때문에 범주형(카테고리) 데이터로 인식하여 인코딩해야한다. one-hot-encoding 방식을 사용한다. 이는 해당하는 열만 1이고 나머지는 0의 값을 가지는 것을 말한다.

In [93]:
pclass_train_dummies = pd.get_dummies(train_df['Pclass'], prefix = 'Pclass') # prefix를 사용하여 열 이름을 설정해주었다.
pclass_test_dummies = pd.get_dummies(test_df['Pclass'], prefix = 'Pclass')

# inplace가 True이면 복사본을 반환하는 것이 아니라 현재의 데이터프레임을 바로 변경하는 것이다. 
# 즉, 데이터프레임에서 Pclass 열을 삭제하는 것이다.
train_df.drop(['Pclass'], axis = 1, inplace = True) 
test_df.drop(['Pclass'], axis = 1, inplace = True)

# join 메서드를 활용하여 열을 추가한다.
train_df = train_df.join(pclass_train_dummies)
test_df = test_df.join(pclass_test_dummies)

In [94]:
train_df

,Survived,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Pclass_1,Pclass_2,Pclass_3
0,0,male,22.0,1,0,7.2500,NaN,S,0,0,1
1,1,female,38.0,1,0,71.2833,C85,C,1,0,0
2,1,female,26.0,0,0,7.9250,NaN,S,0,0,1
3,1,female,35.0,1,0,53.1000,C123,S,1,0,0
4,0,male,35.0,0,0,8.0500,NaN,S,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
886,0,male,27.0,0,0,13.0000,NaN,S,0,1,0
887,1,female,19.0,0,0,30.0000,B42,S,1,0,0
888,0,female,NaN,1,2,23.4500,NaN,S,0,0,1
889,1,male,26.0,0,0,30.0000,C148,C,1,0,0


## 결과 해석
데이터프레임의 Pclass 값이 정수이지만 이는 분류별 정수이기 때문에 범주형 데이터로 변경해주었다. 따라서 Pclass 열을 one-hot-encoding한 것을 만들고 원래의 데이터프레임에서 Pclass 열을 삭제하고 Pclass_1, Pclass_2, Pclass_3 열을 생성하였다.

# 2. sex 데이터 처리하기
- Pclass와 마찬가지로 범주형 데이터에 해당한다.
- 현재 male, female로 되어 있어 이를 one-hot-encoding하여 처리한다.
- 비어있는 데이터가 없다.

In [95]:
sex_train_dummies = pd.get_dummies(train_df['Sex'])
sex_test_dummies = pd.get_dummies(test_df['Sex'])

sex_train_dummies.columns = ['Female', 'Male']
sex_test_dummies.columns = ['Female', 'Male']

train_df.drop(['Sex'], axis = 1, inplace = True)
test_df.drop(['Sex'], axis = 1, inplace = True)

train_df = train_df.join(sex_train_dummies)
test_df = test_df.join(sex_test_dummies)

In [96]:
sex_train_dummies

,Female,Male
0,0,1
1,1,0
2,1,0
3,1,0
4,0,1
...,...,...
886,0,1
887,1,0
888,1,0
889,0,1


위를 보면 알 수 있듯이 여자면 Female에 1 Male에 0, 남자면 Female에 0 Male에 1로 설정된다.

In [97]:
train_df

,Survived,Age,SibSp,Parch,Fare,Cabin,Embarked,Pclass_1,Pclass_2,Pclass_3,Female,Male
0,0,22.0,1,0,7.2500,NaN,S,0,0,1,0,1
1,1,38.0,1,0,71.2833,C85,C,1,0,0,1,0
2,1,26.0,0,0,7.9250,NaN,S,0,0,1,1,0
3,1,35.0,1,0,53.1000,C123,S,1,0,0,1,0
4,0,35.0,0,0,8.0500,NaN,S,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,27.0,0,0,13.0000,NaN,S,0,1,0,0,1
887,1,19.0,0,0,30.0000,B42,S,1,0,0,1,0
888,0,NaN,1,2,23.4500,NaN,S,0,0,1,1,0
889,1,26.0,0,0,30.0000,C148,C,1,0,0,0,1


# 3. Age 데이터 처리하기
- 나이는 연속형 데이터이므로 큰 처리가 필요없다.
- NaN 데이터가 일부 있어서 이를 채우는 처리가 필요하다.
- 이번에는 평균값으로 채운다.

In [98]:
# fillna 메소드는 NaN 값을 채우는 메소드
# inplace = True로 데이터를 바로 수정한다.
train_df['Age'].fillna(train_df['Age'].mean(), inplace = True)
test_df['Age'].fillna(test_df['Age'].mean(), inplace = True)

In [99]:
train_df.info()
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Age       891 non-null    float64
 2   SibSp     891 non-null    int64  
 3   Parch     891 non-null    int64  
 4   Fare      891 non-null    float64
 5   Cabin     204 non-null    object 
 6   Embarked  889 non-null    object 
 7   Pclass_1  891 non-null    uint8  
 8   Pclass_2  891 non-null    uint8  
 9   Pclass_3  891 non-null    uint8  
 10  Female    891 non-null    uint8  
 11  Male      891 non-null    uint8  
dtypes: float64(2), int64(3), object(2), uint8(5)
memory usage: 46.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Age          418 non-null    float64
 2 

위를 보면 알 수 있듯이 Age의 값이 891개로 NaN 값이 없다는 것을 알 수 있다.


# 4. sibSp & Panch  데이터 처리
* SibSp은 sibling(brother, sister, stepbrother, stepsister), spouse(husband, wife)의 인원을 표현한다.
* parch는 parent(mother, father), children(daughter, son, stepdaughter, stepson)의 인원을 표현한다. 이때 몇몇 아이들은 유모와 함께 탔는데 이때는 0으로 처리한다.

> 즉 따로 처리하지 않는다.

# 5. Fare 데이터 처리
* 탑승료를 의미한다.
* 1개의 데이터만이 test 데이터프레임에서 비어있다.
* 비어있는 데이터를 무단 탑승이라고 가정하여 0으로 채운다.

In [100]:
test_df['Fare'].fillna(0, inplace = True)

In [101]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Age          418 non-null    float64
 2   SibSp        418 non-null    int64  
 3   Parch        418 non-null    int64  
 4   Fare         418 non-null    float64
 5   Cabin        91 non-null     object 
 6   Embarked     418 non-null    object 
 7   Pclass_1     418 non-null    uint8  
 8   Pclass_2     418 non-null    uint8  
 9   Pclass_3     418 non-null    uint8  
 10  Female       418 non-null    uint8  
 11  Male         418 non-null    uint8  
dtypes: float64(2), int64(3), object(2), uint8(5)
memory usage: 21.7+ KB


위의 결과를 보면 알수 있듯이 Fare가 418개로 비어있는 값이 없다.

# Cabin 데이터 처리
* 객실 번호를 의미한다.
* 비어있는 값이 아주 많으므로 데이처를 버리기로 한다.

In [102]:
train_df = train_df.drop(['Cabin'], axis = 1)
test_df = test_df.drop(['Cabin'], axis = 1)

In [103]:
train_df.info()
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Age       891 non-null    float64
 2   SibSp     891 non-null    int64  
 3   Parch     891 non-null    int64  
 4   Fare      891 non-null    float64
 5   Embarked  889 non-null    object 
 6   Pclass_1  891 non-null    uint8  
 7   Pclass_2  891 non-null    uint8  
 8   Pclass_3  891 non-null    uint8  
 9   Female    891 non-null    uint8  
 10  Male      891 non-null    uint8  
dtypes: float64(2), int64(3), object(1), uint8(5)
memory usage: 42.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Age          418 non-null    float64
 2   SibSp        418 non-null    int64  


위를 보면 Cabin 열이 삭제 되었다는 것을 알 수 있다.

# Embarked 데이터 처리
* 탑승 항수를 의미한다.
* C = Cherbourg, Q = Queentown, S = Southampton
* 개수를 보았을 때 S가 대다수이므로 빈 값을 S로 채우기로 한다.
* 범주형 데이터 이므로 one-hot-encoding으로 값을 처리한다.

In [107]:
train_df['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [108]:
test_df['Embarked'].value_counts()

S    270
C    102
Q     46
Name: Embarked, dtype: int64

In [109]:
train_df['Embarked'].fillna('S', inplace = True)
test_df['Embarked'].fillna('S', inplace = True)

In [110]:
# Embarked one-hot-encoding 데이터프레임 만들기
embarked_train_dummies = pd.get_dummies(train_df['Embarked'])
embarked_test_dummies = pd.get_dummies(test_df['Embarked'])

# 열 이름 설정
embarked_train_dummies.columns = ['S', 'C', 'Q']
embarked_test_dummies.columns = ['S', 'C', 'Q']

# 기존 Embarded 열 삭제
train_df.drop(['Embarked'], axis = 1, inplace = True)
test_df.drop(['Embarked'], axis = 1, inplace = True)

# 새로 만든 데이터프레임 합치기
train_df = train_df.join(embarked_train_dummies)
test_df = test_df.join(embarked_test_dummies)

#  머신러닝 알고리즘 적용하기

머신러닝 알고리즘을 적용하기 위해서 학습용 데이터를 (정보, 생존 여부)로 나눈다.

In [113]:
X_train = train_df.drop('Survived', axis = 1)
Y_train = train_df['Survived']

# copy 메소드 사용
X_test = test_df.drop('PassengerId', axis = 1).copy()

## 로지스틱 회귀 

In [114]:
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)

Y_pred = logreg.predict(X_test)

logreg.score(X_train, Y_train)

C:\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.8058361391694725

## 서포트 벡터 머신

In [116]:
svc = SVC()
 
svc.fit(X_train, Y_train)

Y_pred = svc.predict(X_test)

svc.score(X_train, Y_train)

0.6868686868686869

In [122]:
! git init

Initialized empty Git repository in C:/Users/jslee/Desktop/�븣怨좊━利� 怨듬��/Kaggle_practice/Titanic/.git/
